# Combiom: searching for combinatorial biomarkers

Algorithm to discover new *combinatorial biomarkers* using linear and kernel ridge regression, Theil-Sen estimator and machine learning.

## Importing packages

In [4]:
import itertools as it
import sklearn.linear_model as sk_lm
import sklearn.preprocessing as sk_pr
import sklearn.kernel_ridge as sk_kr

import pandas as pd
import csv
import numpy as np
import re
import string as st
import numexpr as ne

## Combiom Class

In [29]:
class Combiom:
    """Combiom class"""

    def __init__(self, pars, obs, vols = None):

        if not all(isinstance(a, (float, int)) 
                   for a in [pars, obs, vols]):
            raise ValueError('Please define class arguments: they must be integers and coinсide with data shape')

        self.parameters = int(pars)
        self.observations = int(obs)
        self.volunteers = int(vols)

        self.iterators = {
            'a': range(self.parameters),
            '1/a': range(self.parameters),
            'a/b': it.permutations(range(self.parameters), 2),
            'a/b/c': iter((x, y, z) for x, y, z in it.permutations(range(self.parameters), 3) if z > y),
            'a*b/c': it.combinations(range(self.parameters), 3),
            'a*b/c/d': iter((a, b, c, d) for a, b, c, d in it.permutations(range(self.parameters), 4) if b > a and d > c)
        }


    def load_target_data(self, target_names, target_data):
        '''def load_target_data(self, target_names, target_data)'''

        self.target_names = target_names
        self.target_data = target_data
        self.target_num = len(target_names)


    def load_marker_data(self, marker_names, marker_data):
        '''def load_marker_data(self, marker_names, marker_data):'''

        self.marker_names = marker_names
        self.marker_data = marker_data

    
    def search(self, it_type='all'):

        results = []
        queue = self.iterators.keys() if it_type == 'all' else it_type

        for i in queue:

            r = self.__iterate_combinations(i)
            results.append(r)

        print('Search was successfully finished')
        return results


    def to_dataframe(self, data):
        '''Data must be output of .search() method'''
        
        dfl = []

        # Importing results and adding to a list
        for i in data:
            dfl.append(pd.DataFrame(i, columns=['Biomarker', 'Target marker',
                                                'M1', 'M2', 'M3', 'M4',
                                                'TS', 'R', 'KR', 'MID', 'TID', 'Type']))
        
        df = pd.concat(dfl)
        df = df.rename(columns={'TS': 'Theil-Sen Score', 'R': 'Ridge Score', 'KR': 'Kernel Ridge Score',
                                'M1': 'Marker 1', 'M2': 'Marker 2', 'M3': 'Marker 3', 'M4': 'Marker 4'})
        
        return df


    def __iterate_combinations(self, iterator_type):

        p_num = self.parameters
        p_names = self.marker_names
        p_data = self.marker_data
        t_num = self.target_num
        t_names = self.target_names
        t_data = self.target_data

        output = {'Biomarker': [], 'Target marker': [],
                  'M1': [], 'M2': [], 'M3': [], 'M4': [],
                  'TS': [], 'R': [], 'KR': [],
                  'MID': [], 'TID': [], 'Type': []}

        print('Search:', iterator_type, 'started')
        
        for a, t in it.product(self.iterators[iterator_type], range(t_num)):

            list_ids = np.array([a]).ravel()
            list_ops = list(re.sub(r"\w", "", iterator_type))

            # Naming a combinatorial marker
            marker_name = self.__name(iterator_type, list_ids)

            # Naming simple markers and padding list with np.nan to make it 4-element
            simple_markers_names = np.pad(self.marker_names[[list_ids]].astype('object'),
                                          (0, 4-len(list_ids)), mode='constant', constant_values=(0, np.nan))
            
            # Calculating marker
            marker_value = self.__calc_marker(iterator_type, list_ids)

            # Preprocessing data: transforming NaN values into neighbours-mean
            # and normalizing data 
            marker_value = self.__transform_nan(marker_value)
            marker_value_norm = self.__normalize(marker_value)

            # Marker IDs joining into a string
            mid = ', '.join(map(str, list_ids))

            # Naming a target
            target_name = t_names[t]
            target_data = np.copy(t_data[t].reshape(-1, 1))
            tid = str(t)

            # Regression
            ts_score, ridge_score, kr_score = self.__regression(marker_value_norm, target_data)

            # Processing results
            if any(z > 0.8 for z in (np.around(ts_score, 1), np.around(ridge_score, 1), np.around(kr_score, 1))):

                for n, v in zip(['Biomarker', 'Target marker',
                                 'M1', 'M2',
                                 'M3', 'M4',
                                 'TS', 'R', 'KR', 'MID', 'TID', 'Type'],
                                [marker_name, target_name,
                                 simple_markers_names[0], simple_markers_names[1],
                                 simple_markers_names[2], simple_markers_names[3],
                                 ts_score, ridge_score, kr_score, mid, tid, iterator_type]):

                    output[n].append(v)

        print('Search:', iterator_type, 'finished')

        return output

    def __regression(self, x, y, regressor_return=False):

        # TheilSen Regression
        ts_y = np.copy(y.ravel())
        ts = sk_lm.TheilSenRegressor()
        ts.fit(x, ts_y)

        # r squared
        ts_y_pred = ts.predict(x)
        ts_y_mean = np.mean(ts_y)
        ts_ssr = np.sum((ts_y_pred - ts_y_mean) ** 2)
        ts_sst = np.sum((ts_y - ts_y_mean) ** 2)
        #ts_score = np.absolute(ts.score(index_norm, ts_y))
        ts_score = ts_ssr / ts_sst

        # Ridge Regression
        # Normalizating & reshaping
        ridge = sk_lm.Ridge(alpha=0.01, normalize=False)
        ridge.fit(x, y)
        ridge_score = np.absolute(ridge.score(x, y))

        # Kernel Ridge Regression
        kernel_ridge = sk_kr.KernelRidge(kernel='rbf', alpha=0.0001)
        kernel_ridge.fit(x, y)
        kr_score = np.absolute(kernel_ridge.score(x, y))

        if regressor_return:
            return (ts, ridge, kernel_ridge)
        else:
            return (ts_score, ridge_score, kr_score)



    def __transform_nan(self, x, strategy='mean'):

        imp = sk_pr.Imputer(missing_values='NaN', strategy=strategy)
        x = imp.fit_transform(x.reshape(-1, 1))

        return x


    def __normalize(self, x):

        return sk_pr.normalize(x, axis=0)


    def __name(self, marker_type, ids):

        data = {}
        for l, p in zip(st.ascii_letters[: ids.size], self.marker_names[ids]):
            data[l] = p

        rep = dict((re.escape(k), v) for k, v in data.items())
        pattern = re.compile("|".join(rep.keys()))
        text = pattern.sub(lambda m: rep[re.escape(m.group(0))], marker_type)

        return text


    def __calc_marker(self, marker_type, ids):

        data = {}

        for l, p in zip(st.ascii_letters[: ids.size], self.marker_data[ids]):
            data[l] = p

        # Example: a*b/c (type), {a: ..., b: ..., c: ...} (data)
        marker = ne.evaluate(marker_type, data)
        
        return marker


    def retrain(self, marker_names, marker_type, target_name):

        marker_ids = []

        for n in markers_names:
            marker_ids.append(np.where(self.marker_names == n)[0][0])

        target_data = self.target_data[np.where(self.target_names == target_name)[0][0]]

        d = {}
        for i, (l, p) in enumerate(zip(st.ascii_letters[: len(marker_ids)], self.marker_data[[marker_ids]])):
            d[l] = p

        x = ne.evaluate(marker_type, d)

        # Calculating marker
        marker_value = self.__calc_marker(marker_type, marker_ids)

        # Preprocessing data: transforming NaN values into neighbours-mean
        # and normalizing data 
        marker_value = self.__transform_nan(marker_value)
        marker_value_norm = self.__normalize(marker_value)

        # regression
        ts, ridge, kernel_ridge = self.__regression(index_norm, targetdata, regressor_return=True)
        return (ts, ridge, kernel_ridge)


## 1 Biochemical Dataset
### 1.1 Importing and preprocessing

In [30]:
# Parameter names
bio_csv_names = csv.reader(open('./BiochemData/parameters-names.csv'), delimiter=';')
bio_marker_names = np.array(list(bio_csv_names)).flatten()

# Setting shape of data array
bio_volunteers = 10
bio_parameters = 11
bio_points = 8

# Initializing data array with zeros
bio_data = np.zeros(shape=(bio_volunteers, bio_parameters, bio_points))

# Timepoints at which each biochemical parameter was measured (in hours after experiment)
# Null stands for "before experiment", 1 - in an hour after experiment, so on...
bio_timepoints = np.array([0, 1, 1*24, 2*24, 3*24, 5*24, 7*24, 9*24])

# Reading numeric data from csv-files. One file per participant
# Each file has a shape of N x M
#     where N is the number of parameters and M is the number of timepoints at which they were measured
for i in np.arange(bio_volunteers):
    bio_csv_file = csv.reader(open('./BiochemData/%d.csv' % i), delimiter=';')
    bio_data[i, 0: bio_parameters + 1, 0: bio_points + 1] = np.array(list(bio_csv_file), np.float64)

# Shape of data array
np.shape(bio_data)

(10, 11, 8)

### 1.2 Target markers

In [31]:
# Reading data for creatine kinase
bio_csv_ck = csv.reader(open('./BiochemData/creatine-kinase.csv'), delimiter=';')
bio_ckdata = np.array(list(bio_csv_ck), np.float64)

# Reading data for aspartate transferase
bio_csv_ast = csv.reader(open('./BiochemData/aspartate-transferase.csv'), delimiter=';')
bio_astdata = np.array(list(bio_csv_ast), np.float64)

# Reading data for myoglobin
bio_csv_mg = csv.reader(open('./BiochemData/myoglobin.csv'), delimiter=';')
bio_mgdata = np.array(list(bio_csv_mg), np.float64)
bio_mgdata[bio_mgdata == 0] = np.nan

imp = sk_pr.Imputer(missing_values='NaN', strategy='mean')
bio_mgdata = imp.fit_transform(bio_mgdata)

# Reading data for albumin
bio_csv_alb = csv.reader(open('./BiochemData/albumin.csv'), delimiter=';')
bio_albdata = np.array(list(bio_csv_alb), np.float64)
bio_albdata[bio_albdata == 0] = np.nan

# Processing creatinekinase and AST data
# Making an array of maximum values and taking a LOG10 element-wisely
bio_ckmax = np.log10([np.max(z) for z in bio_ckdata])
bio_astmax = np.log10([np.max(z) for z in bio_astdata])
bio_mgmax = np.log10([np.max(z) for z in bio_mgdata])

bio_alb = np.log10(bio_albdata[np.arange(bio_albdata.shape[0]), [np.argmax(z) for z in bio_ckdata]])

bio_ckmg = bio_ckmax / np.log10(bio_mgdata[np.arange(bio_mgdata.shape[0]), [np.argmax(z) for z in bio_ckdata]])
bio_ckast = bio_ckmax / np.log10(bio_astdata[np.arange(bio_astdata.shape[0]), [np.argmax(z) for z in bio_ckdata]])
bio_ckalb = bio_ckmax / bio_alb
bio_mgalb = bio_mgmax / bio_alb

# Target markers
bio_target_names = np.array(['Creatinekinase', 'AST', 'Myoglobin',
                             'Creatinekinase / AST', 'Creatinekinase / Myoglobin',
                             'Creatinekinase / Albumin', 'Myoglobin / Albumin'])

bio_target_data = np.vstack((bio_ckmax, bio_astmax, bio_mgmax, bio_ckmg, bio_ckast, bio_ckalb, bio_mgalb))

# Making an array of 1-hour measures
bio_marker_data = bio_data[:,:,1].T
np.shape(bio_marker_data)

(11, 10)

### 1.3 Searching

In [32]:
# Initialize class with 11 parameters, 8 observations and 10 participants
cmb = Combiom(11, 8, 10)

cmb.load_marker_data(bio_marker_names, bio_marker_data)
cmb.load_target_data(bio_target_names, bio_target_data)

bio_results = cmb.search('all')

Search: a started
Search: a finished
Search: a*b/c/d started
Search: a*b/c/d finished
Search: a/b/c started
Search: a/b/c finished
Search: a*b/c started
Search: a*b/c finished
Search: a/b started
Search: a/b finished
Search: 1/a started
Search: 1/a finished
Search was successfully finished


### 1.4 Results. Exporting to Pandas and Pickle

In [33]:
# Creating DataFrame
bio_df = cmb.to_dataframe(bio_results)

# Export to Pickle file
bio_df.to_pickle('./biochem_db.pickle')

### 1.5 Exporting to Excel

#### Top combinatorial biochemical markers which correlate with each target marker

In [39]:
biochem_max_group = bio_df.groupby(('Target marker'))\
                          [['Biomarker', 'Theil-Sen Score', 'Ridge Score', 'Kernel Ridge Score']]\
                          .max().sort_values(by=['Kernel Ridge Score'], ascending=False)
biochem_max_group

,Biomarker,Theil-Sen Score,Ridge Score,Kernel Ridge Score
Target marker,,,,
Creatinekinase / AST,Uric acid/TAG/MCFA,1.387498,0.803509,0.945346
Creatinekinase,Uric acid/Urea/TAG,8.745679,0.830929,0.923575
Creatinekinase / Albumin,Uric acid/Urea/TAG,7.764427,0.833813,0.917624
AST,Uric acid*MCFA/Urea/TAG,7.336094,0.846807,0.903691
Myoglobin,Uric acid/Urea/TAG,6.034839,0.812799,0.872165
Myoglobin / Albumin,Uric acid/Chloride,6.013822,0.810361,0.869824
Creatinekinase / Myoglobin,Uric acid*MCFA/Urea/Lactate,7.029963,0.319701,0.785268


In [51]:
# Initialize Pandas Excel writer with XlsxWriter
writer = pd.ExcelWriter('./combinatorial_biochemical_biomarkers.xlsx', engine='xlsxwriter')

# Convert Dataframe to Excel object
biochem_max_group.to_excel(writer, sheet_name='Target Max')

#### Combinatorial biomarkers sorted by the number of target markers they correlate with.

In [52]:
biochem_max_top = bio_df.groupby(('Biomarker'))\
                        .agg({'Target marker': 'count', 'Kernel Ridge Score': 'mean', 'Theil-Sen Score': 'mean', 'Ridge Score': 'mean'})\
                        .sort_values(['Target marker', 'Kernel Ridge Score'], ascending=False)\
                        .head(10)\
                        [['Target marker', 'Kernel Ridge Score', 'Ridge Score', 'Theil-Sen Score']]\
                        .reset_index()
biochem_max_top

,Biomarker,Target marker,Kernel Ridge Score,Ridge Score,Theil-Sen Score
0,Urea*Phosphate/Lactate/Bilirubin,6,0.792913,0.259419,1.669345
1,Creatinine*Urea/Lactate/Bilirubin,6,0.678736,0.246219,1.534484
2,Cholesterol*Urea/Lactate/Bilirubin,6,0.670188,0.230477,1.219462
3,Urea*Glucose/Uric acid/MCFA,5,0.862664,0.653679,0.918536
4,Creatinine*Phosphate/Lactate/Uric acid,5,0.810566,0.321485,1.897554
5,Uric acid*MCFA/Urea/Glucose,5,0.810290,0.562593,1.588352
6,Urea*Phosphate/Lactate/MCFA,5,0.807465,0.381844,1.436135
7,Phosphate/Lactate/Uric acid,5,0.799591,0.346011,1.765379
8,Phosphate*Chloride/Lactate/Uric acid,5,0.795285,0.356422,1.635243
9,Creatinine*Urea/Lactate/MCFA,5,0.791425,0.350083,1.057615


In [53]:
# Convert the dataframe to an XlsxWriter Excel object.
biochem_max_top.to_excel(writer, sheet_name='Overall Max')

#### Combinatorial biomarkers sorted by the number of target markers they correlate with.

In [54]:
biochem_max_type = bio_df.groupby('Type')[['Biomarker', 'Theil-Sen Score', 'Ridge Score', 'Kernel Ridge Score']]\
                          .max().sort_values(by=['Kernel Ridge Score'], ascending=False)
                         
biochem_max_type

,Biomarker,Theil-Sen Score,Ridge Score,Kernel Ridge Score
Type,,,,
a/b/c,Uric acid/Urea/TAG,3.046988,0.836749,0.945346
a*b/c/d,Uric acid*MCFA/Urea/TAG,8.745679,0.846807,0.934558
a/b,Uric acid/Phosphate,1.810922,0.771610,0.879388
a*b/c,Urea*Phosphate/MCFA,6.636919,0.753447,0.835570
a,Uric acid,1.259103,0.517416,0.683235
1/a,1/Uric acid,1.868534,0.402234,0.669841


In [55]:
# Convert the dataframe to an XlsxWriter Excel object.
biochem_max_type.to_excel(writer, sheet_name='Type Max')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

## 2 Physiological Dataset
### 2.1 Importing and preprocessing

In [57]:
# Parameter names
phys_csv_names = csv.reader(open('./PhysData/parameters-names.csv'), delimiter=';')
phys_marker_names = np.array(list(phys_csv_names)).ravel()

# Setting shape of data array
phys_volunteers = 10
phys_parameters = 15
phys_points = 8

# Initializing data array with zeros
phys_data = np.zeros(shape=(phys_volunteers, phys_parameters, phys_points))

# Timepoints at which each biochemical parameter was measured (in hours after experiment)
# 0 stands for "before experiment", 0.05 - shortly after experiment, 1 - in an hour after experiment, so on...
phys_timepoints = np.array([0, 0.05, 1, 1*24, 2*24, 3*24, 5*24, 7*24, 9*24])

# Reading numeric data from csv-files
# Each file has a shape of N x M
#     where N is the number of parameters and M is the number of timepoints at which they were measured
for i in np.arange(phys_volunteers):
    phys_csv_file = csv.reader(open('./PhysData/%d.csv' % i), delimiter=';')
    phys_data[i, 0: phys_parameters + 1, 0: phys_points + 1] = np.array(list(phys_csv_file), np.float64)

# Mask zeros with NaN
phys_data[phys_data == 0] = np.nan

# Making an array of 1-hour measures
phys_marker_data = phys_data[:,:,1].T

# Shape
np.shape(phys_marker_data)

(15, 10)

### 2.2 Search

In [58]:
# Initialize class with 15 parameters, 8 observations and 10 participants
cmbp = Combiom(15, 8, 10)

cmbp.load_marker_data(phys_marker_names, phys_marker_data)
cmbp.load_target_data(bio_target_names, bio_target_data)

phys_results = cmbp.search()

Search: a started
Search: a finished
Search: a*b/c/d started
Search: a*b/c/d finished
Search: a/b/c started
Search: a/b/c finished
Search: a*b/c started
Search: a*b/c finished
Search: a/b started
Search: a/b finished
Search: 1/a started
Search: 1/a finished
Search was successfully finished


### 2.3 Results. Export to DataFrame and Pickle

In [60]:
# Creating DataFrame
phys_df = cmbp.to_dataframe(phys_results)

# Export to Pickle file
phys_df.to_pickle('./phys_db.pickle')

### 2.4 Exporting to Excel

#### Top combinatorial physiological markers which correlate with each target marker

In [63]:
phys_max_group = phys_df.groupby(('Target marker'))\
                       [['Biomarker', 'Theil-Sen Score', 'Ridge Score', 'Kernel Ridge Score']]\
                       .max().sort_values(by=['Kernel Ridge Score'], ascending=False)
phys_max_group

,Biomarker,Theil-Sen Score,Ridge Score,Kernel Ridge Score
Target marker,,,,
Creatinekinase / AST,SYS Blood Pres 1/H-r Length/Contrac Tone,3.656179,0.826668,0.958987
Creatinekinase / Myoglobin,SYS Blood Pres 2/H-r Length/Isometric Strength,9.272325,0.532325,0.909044
Creatinekinase / Albumin,SYS Blood Pres 2/Max Amp EMG/Isometric Strength,6.844295,0.629212,0.861057
Creatinekinase,SYS Blood Pres 2/Max Amp EMG/Isometric Strength,6.265478,0.611061,0.860031
AST,SYS Blood Pres 2/Isometric Strength/SYS Blood ...,5.638955,0.615336,0.792680
Myoglobin / Albumin,SYS Blood Pres 2/Max Amp EMG,8.197722,0.620315,0.786178
Myoglobin,SYS Blood Pres 2/Max Amp EMG,8.183847,0.613169,0.771977


In [64]:
# Initialize Pandas Excel writer with XlsxWriter
writer = pd.ExcelWriter('./combinatorial_physiological_biomarkers.xlsx', engine='xlsxwriter')

# Convert Dataframe to Excel object
phys_max_group.to_excel(writer, sheet_name='Target Max')

#### Combinatorial biomarkers sorted by the number of target markers they correlate with.

In [65]:
phys_max_top = phys_df.groupby(('Biomarker'))\
                      .agg({'Target marker': 'count', 'Kernel Ridge Score': 'mean', 'Theil-Sen Score': 'mean', 'Ridge Score': 'mean'})\
                      .sort_values(['Target marker', 'Kernel Ridge Score'], ascending=False)\
                      .head(10)\
                      [['Target marker', 'Kernel Ridge Score', 'Ridge Score', 'Theil-Sen Score']]\
                      .reset_index()
phys_max_top

,Biomarker,Target marker,Kernel Ridge Score,Ridge Score,Theil-Sen Score
0,H-r Length*Contrac Tone/Max Amp EMG/Mean Amp EMG,6,0.484344,0.199650,2.468996
1,H-r Length*Contrac Tone/Max Amp EMG/L-Thigh Circ,6,0.467022,0.218022,1.281292
2,R-Thigh Circ*DIA Blood Pres 2/Contrac Tone/SYS...,6,0.464185,0.369479,0.989356
3,H-r Length/Max Amp EMG,6,0.259930,0.163553,1.041286
4,H-r Length/Max Amp EMG/R-Thigh Circ,6,0.243247,0.162328,2.026401
5,Contrac Tone/Isometric Strength/DIA Blood Pres 1,5,0.812543,0.260911,4.224298
6,R-Thigh Circ*Contrac Tone/Isometric Strength/D...,5,0.752616,0.306047,2.361174
7,Contrac Tone/L-Thigh Circ/Isometric Strength,5,0.723886,0.266356,2.410329
8,Contrac Tone/Max Amp EMG/Isometric Strength,5,0.699804,0.282086,1.985587
9,H-r Length*Contrac Tone/Mean Amp EMG/DIA Blood...,5,0.661110,0.184189,5.337367


In [66]:
# Convert the dataframe to an XlsxWriter Excel object.
phys_max_top.to_excel(writer, sheet_name='Overall Max')

#### Combinatorial biomarkers sorted by the number of target markers they correlate with

In [67]:
phys_max_type = phys_df.groupby('Type')[['Biomarker', 'Theil-Sen Score', 'Ridge Score', 'Kernel Ridge Score']]\
                       .max().sort_values(by=['Kernel Ridge Score'], ascending=False)
                         
phys_max_type

,Biomarker,Theil-Sen Score,Ridge Score,Kernel Ridge Score
Type,,,,
a*b/c/d,SYS Blood Pres 2*DIA Blood Pres 2/Contrac Tone...,9.272325,0.793042,0.958987
a*b/c,Relax Tone*Contrac Tone/Isometric Strength,1.364161,0.612786,0.947877
a/b/c,SYS Blood Pres 2/Max Amp EMG/Isometric Strength,8.197722,0.826668,0.909044
a/b,SYS Blood Pres 2/Max Amp EMG,1.383819,0.441696,0.908489
1/a,1/Max Amp EMG,1.164506,0.229497,0.334895
a,SYS Blood Pres 1,1.023531,0.064495,0.148289


In [68]:
# Convert the dataframe to an XlsxWriter Excel object.
phys_max_type.to_excel(writer, sheet_name='Type Max')

# Close the Pandas Excel writer and output the Excel file.
writer.save()